In [ ]:
from math import factorial
import matplotlib.pyplot as plt
import numpy as np

def offeredTraffic(calls_per_hour, hours_per_call): # A0
    return calls_per_hour*hours_per_call

def erlangB(n, A0):
    denom = 0
    for i in range(n+1):
        denom += (A0**i)/(factorial(i))
    E1 = ((A0**n)/factorial(n))/denom
    return E1

# def erlangC(n, A0):
#     denom = ((A0**n)/factorial(n))*(n/(n-A0))
#     for i in range(n):
#         denom += (A0**i)/factorial(i)
#     return ((A0**n)/factorial(n))*(n/(n-A0))/denom
def erlangC(n, A0):
    denomSum = 0
    for i in range(n):
        denomSum += (A0**i)/factorial(i)
    denom = A0**n + factorial(n)*(1-A0/n)*denomSum
    return (A0**n)/denom

def meanCallDelay(n, A0, meanCallDuration, probDelay):
    return probDelay*meanCallDuration/(n-A0)

def meanQueuedDelay(n, A0, meanCallDuration):
    return meanCallDuration/(n-A0)

    
# def erlangC_GOS():



In [ ]:
def trafficSimulation(numChannels, numCalls, meanCallDuration, requeue=False):

    callDurations = np.random.exponential(scale=meanCallDuration, size=numCalls)
    callStarts = np.random.uniform(size=numCalls)
    
    callStarts.sort()
    
    # Calls = Numpy array of [[call0_start call0_end]; [call1_start call1_end];...
    calls = np.stack((callStarts, np.add(callStarts,callDurations)),axis=1)

    channelFreeTimes = np.zeros(numChannels)
    
    time = 0
    callsFailed = 0
    for i, call in enumerate(calls):
        time = call[0]
        channelFound = False
        for j in range(numChannels):
            if not(channelFound):
                if channelFreeTimes[j] < time:
                    # print("Time:", time)
                    # print("channelFreeTimes", channelFreeTimes)
                    # print("Channel {} free. Adding call time {}".format(j, call[1]))
                    channelFreeTimes[j] = call[1]
                    channelFound = True
        if channelFound == False:
            callsFailed += 1
            # print("Call {} ({}) failed.\n".format(i,call))
            # if requeue:


    return callsFailed/numCalls, callsFailed


In [ ]:
meanCallDuration = 180/3600 # 3 mins in hrs
n = 11

minNumCalls = 1
maxNumCalls = 2000
numTests = 21
# simulationCallAmounts = np.concatenate((np.linspace(minNumCalls, maxNumCalls, numTests), np.linspace(2500, 6000, 8))).astype(np.int)
# simulationCallAmounts = np.linspace(100, 2000, 20).astype(np.int)
simulationCallAmounts = np.linspace(200, 200, 1).astype(np.int)

erlangBs = np.zeros(simulationCallAmounts.shape)
erlangCs = np.zeros(simulationCallAmounts.shape)
GOS_simulated = np.zeros(simulationCallAmounts.shape)

repetitionsPerSimulation = 10
print(simulationCallAmounts)

In [ ]:
for i, numCalls in enumerate(simulationCallAmounts):
    # print("{} - For {} calls in a network with {} channels:".format(i, numCalls, n))
    A0 = offeredTraffic(numCalls, meanCallDuration)
    erlangBs[i] = erlangB(n, A0)*100
    erlangCs[i] = erlangC(n, A0)*100
    # print("Erlang B = {}".format(GOS_erlang[i]))

    totalFailRate, totalFailCount = 0, 0
    for j in range(repetitionsPerSimulation):
        failRate, failsCount = trafficSimulation(n, numCalls, meanCallDuration)
        totalFailRate += failRate
        totalFailCount += failsCount
    meanFailRate = totalFailRate/repetitionsPerSimulation
    meanFailCount = totalFailCount/repetitionsPerSimulation
    GOS_simulated[i] = meanFailRate*100
    # print("Call fail rate in {} simulations = {}% (mean of {} fails in {} calls)\n".format(repetitionsPerSimulation, GOS_simulated[i], meanFailCount, numCalls))

print("# Calls:\tErlang B:\tErlang C:\tSimulation:")
for i, numCalls in enumerate(simulationCallAmounts):
    print("{}\t\t{:.3f}\t\t{:.3f}\t\t{:.3f}".format(numCalls,erlangBs[i], erlangCs[i], GOS_simulated[i]))

In [ ]:
# print("Erlang B:\t\t\t Simulation:")
# for i, numCalls in enumerate(simulationCallAmounts):
#     print(GOS_erlang[i], "\t\t", GOS_simulated[i])

plt.plot(simulationCallAmounts, erlangBs, label='Erlang B')
plt.plot(simulationCallAmounts, np.clip(erlangCs,0,100), label='Erlang C')
plt.plot(simulationCallAmounts, GOS_simulated, label='Exponential')
plt.xlabel('Number of calls')
plt.ylabel('Grade Of Service (%)')
plt.title("GOS vs number of calls made")
plt.legend()
plt.show()